In [1]:
import pandas as pd
import zipfile
import gc  # For garbage collection

# Initialize counters for each label
label_counts = {
    "Fe-Oxide": 0,
    "Ilmenite": 0,
    "Epidote": 0,
    "AlFe-Silicate": 0,
    "Amphibole": 0,
    "Background": 0
    
}

desired_samples_per_label = 10000
samples = []  # To store the extracted samples

max_samples_per_file_per_label = 5  # Set your desired limit


In [1]:
import dask
import dask.dataframe as dd
import zipfile
import gc
import os
from dask import config

# Limiting and monitoring memory usage
config.set({
    'memory': {
        'target': 0.8,
        'spill': 0.85,
        'pause': 0.9,
        'terminate': 0.95
    }
})

# Set the temporary directory for Dask to 'E:'
dask.config.set(temporary_directory='E:')

# Initialize counters for each label
label_counts = {
    "Fe-Oxide": 0,
    "Ilmenite": 0,
    "Epidote": 0,
    "AlFe-Silicate": 0,
    "Amphibole": 0,
    "Background": 0
}

desired_samples_per_label = 10000
max_samples_per_file_per_label = 50

def save_to_file(accumulated_df, file_num):
    output_file = os.path.join("E:\\", f"combined_data_{file_num}.csv")
    accumulated_df.compute().to_csv(output_file, index=False)
    print(f"Saved to {output_file}")
    return file_num + 1

accumulated_data = []
rows_accumulated = 0
file_num = 1

zip_file_path = "E:\\CH Counts Data.zip"
with zipfile.ZipFile(zip_file_path, 'r') as z:
    total_files = sum(1 for file_name in z.namelist() if file_name.startswith("CH Counts Data/") and file_name.endswith(".csv"))
    current_file = 0

    for file_name in z.namelist():
        if not file_name.startswith("CH Counts Data/") or not file_name.endswith(".csv"):
            continue

        current_file += 1
        print(f"Processing file {current_file}/{total_files}: {file_name}")

        zip_path = f"zip://{zip_file_path}!/{file_name}"
        df = dd.read_csv(zip_path)

        for label, count in label_counts.items():
            if count < desired_samples_per_label:
                label_rows = df[df['Mineral Phase'] == label]
                needed_samples = min(desired_samples_per_label - count, max_samples_per_file_per_label)
                samples_to_add = label_rows.head(needed_samples, npartitions=-1)
                accumulated_data.append(samples_to_add)
                label_counts[label] += len(samples_to_add)
                rows_accumulated += len(samples_to_add)

                if rows_accumulated >= 10000:
                    accumulated_df = dd.concat(accumulated_data)
                    file_num = save_to_file(accumulated_df, file_num)
                    accumulated_data = []
                    rows_accumulated = 0

        del df
        del label_rows
        del samples_to_add
        gc.collect()

        print(f"Current label counts: {label_counts}")

        if all([count >= desired_samples_per_label for count in label_counts.values()]):
            print("All required samples collected.")
            break

if accumulated_data:
    accumulated_df = dd.concat(accumulated_data)
    save_to_file(accumulated_df, file_num)

print("Processing completed.")

Processing file 1/8327: CH Counts Data/Group8_0001.csv


PermissionError: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: [Errno 13] Permission denied: 'C:/Users/61596'